In [4]:
reviews_train = []
for line in open('C:/Users/suhas/Desktop/DM Mini-Project/movie_data/full_train.txt', 'r',encoding='utf8'):
    reviews_train.append(line.strip())

reviews_test = []
for line in open('C:/Users/suhas/Desktop/DM Mini-Project/movie_data/full_test.txt', 'r',encoding='utf8'):
    reviews_test.append(line.strip())

import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")


def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

    return reviews


reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 12500 else 0 for i in range(25000)]

X_train, X_val, y_train, y_val = train_test_split( X, target, train_size=0.75)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print("Accuracy for C=%s: %s"
          % (c, accuracy_score(y_val, lr.predict(X_val))))


final_model = LogisticRegression(C=0.05)
final_model.fit(X, target)
print ("Final Accuracy: %s"
       % accuracy_score(target, final_model.predict(X_test)))

feature_to_coef = {
    word: coef for word, coef in zip(
    cv.get_feature_names(), final_model.coef_[0]
)
}
for best_positive in sorted(
        feature_to_coef.items(),
        key=lambda x: x[1],
        reverse=True)[:5]:
    print(best_positive)

for best_negative in sorted(
        feature_to_coef.items(),
        key=lambda x: x[1])[:5]:
    print(best_negative)


Accuracy for C=0.01: 0.87104
Accuracy for C=0.05: 0.88064
Accuracy for C=0.25: 0.88192
Accuracy for C=0.5: 0.88112
Accuracy for C=1: 0.8784
Final Accuracy: 0.88152
('excellent', 0.9292549193423959)
('perfect', 0.7907005834710253)
('great', 0.6745323523875933)
('amazing', 0.6127040012519653)
('superb', 0.601936804694158)
('worst', -1.3645959239411622)
('waste', -1.166424223235376)
('awful', -1.0324189681376759)
('poorly', -0.8752018837121309)
('boring', -0.856354346809877)
